In [1]:
import torch

def create_distance_matrix(self, atoms, data):
    device = atoms.device
    batch_size = atoms.size(0)
    max_atoms = atoms.size(1)
    
    distances = torch.full((batch_size, max_atoms, max_atoms), 8., device=device)
    
    batch_indices = torch.arange(batch_size, device=device)
    atom_indices = torch.arange(max_atoms, device=device)
    distances[batch_indices.unsqueeze(1), atom_indices, atom_indices] = 0
    
    source, target = data.edge_index
    edge_batch = data.batch[source]
    # Compute offsets for each batch
    offsets = torch.cumsum(torch.cat([torch.zeros(1, device=device), data.n_atoms[:-1]]), dim=0)
    
    source_adj = (source - offsets[edge_batch]).to(torch.long)
    target_adj = (target - offsets[edge_batch]).to(torch.long)        
    distances[edge_batch, source_adj, target_adj] = data.edge_weight
    distances[edge_batch, target_adj, source_adj] = data.edge_weight
    
    return distances

In [ ]:
from torch_geometric.data import Data

atoms = torch.tensor([[2, 1, 2, 0, 0], [1, 1, 2, 2, 3]])
data = Data()
data.edge_index = torch.tensor([[0, 1, 3, 4], [1, 2, 5, 9]])
data.edge_weight = torch.tensor([1., 2., 3., 4.])
data.batch = torch.tensor([0, 0, 0, 1, 1, 1, 1, 1])